In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd

from pathlib import Path

# plt.style.use('seaborn-v0_8-whitegrid')

In [62]:
# https://censusreporter.org

data_raw_path = Path().resolve().parent / "data"
images_path = Path().resolve().parent / "images" / "mobility_segregation"
images_path.mkdir(exist_ok=True)
matrix_path = data_raw_path / "matrix_20240306"
compmean_path = data_raw_path / "compmean_20240306"
tract_tessellation_filepath = data_raw_path / "tl_2019_53_tract.shp"
tract_centroids_filepath = data_raw_path / "tract_indices_centroid.csv"

data_path = Path().resolve() / "data"  # Current folder

In [12]:
drop_tract_list = [
    53033032500,
    53033032800,
    53033032702,
    53033031502,
    53033990100  # no residents
]

In [13]:
tract_tessellation = (
    gpd.read_file(tract_tessellation_filepath)
    .rename(columns={"GEOID": "tract"})
    .astype({"tract": int})
    .query("COUNTYFP == '033'")
    .query("tract not in @drop_tract_list")
    .loc[:, ["tract", "geometry"]]
)
tract_tessellation.head()

,tract,geometry
96,53033025701,"POLYGON ((-122.20643 47.46968, -122.20642 47.4..."
97,53033025702,"POLYGON ((-122.18587 47.46643, -122.1858 47.46..."
98,53033025804,"POLYGON ((-122.18595 47.457, -122.18593 47.458..."
99,53033026001,"POLYGON ((-122.24964 47.49934, -122.24951 47.4..."
100,53033026100,"POLYGON ((-122.27058 47.49558, -122.2703 47.49..."


In [14]:
tract_centroids = (
    pd.read_csv(tract_centroids_filepath)
    .pipe(
        lambda x: gpd.GeoDataFrame(
            x,
            geometry=gpd.points_from_xy(x["centroid_pop_long"], x["centroid_pop_lat"]),
            crs=tract_tessellation.crs
        ) # type: ignore
    )
    .loc[lambda x: x["tract"].isin(tract_tessellation["tract"])]
    .sort_values(by="tract")
)
tract_centroids.head()

,id,tract,pop2010,centroid_pop_lat,centroid_pop_long,geometry
92,93,53033000100,6255,47.726814,-122.291489,POINT (-122.29149 47.72681)
135,136,53033000200,7646,47.726185,-122.308673,POINT (-122.30867 47.72618)
188,189,53033000300,2603,47.730773,-122.335689,POINT (-122.33569 47.73077)
342,343,53033000401,5551,47.728397,-122.352217,POINT (-122.35222 47.7284)
41,42,53033000402,4841,47.717895,-122.354291,POINT (-122.35429 47.7179)


In [ ]:
id_tract_dict = tract_centroids.set_index("id")["tract"].to_dict()
drop_id_list = {k: v for k, v in id_tract_dict.items() if k not in drop_tract_list}
tract_id_dict = {v: k for k, v in id_tract_dict.items()}

tract_tessellation = tract_tessellation.assign(id=lambda x: x["tract"].map(tract_id_dict))
tract_tessellation.head()

,tract,geometry,id
96,53033025701,"POLYGON ((-122.20643 47.46968, -122.20642 47.4...",67
97,53033025702,"POLYGON ((-122.18587 47.46643, -122.1858 47.46...",130
98,53033025804,"POLYGON ((-122.18595 47.457, -122.18593 47.458...",377
99,53033026001,"POLYGON ((-122.24964 47.49934, -122.24951 47.4...",202
100,53033026100,"POLYGON ((-122.27058 47.49558, -122.2703 47.49...",66


In [19]:
n_tracts = tract_tessellation.shape[0]
id_list = tract_tessellation["id"].unique()
tract_list = tract_tessellation["tract"].unique()

## Total devices

devices.csv: Total number of devices in each census block group in the census tract added together across all days in the one-week period starting with the date in the filename.

In [20]:
devices_dict = {}
for f in matrix_path.glob("*devices.csv"):
    month_day, year= f.stem.removesuffix("devices").split("_")
    month, day = month_day.split("-")
    day = day if len(day) == 2 else "0" + day
    file_date = "-".join([year, month, day])
    devices_dict[file_date] = pd.read_csv(
        f,
        index_col=0,
        names=["id", "n_devices"],
        header=0
    )

In [21]:
devices_total = (
    pd.concat(devices_dict, names=["date"])
    .reset_index()
    .query("id in @drop_id_list")
    .assign(
        date=lambda x: pd.to_datetime(x["date"]),
        tract=lambda x: x["id"].map(id_tract_dict)
    )
    .sort_values(["date", "tract"])
)
devices_total.head()

,date,id,n_devices,tract
490,2019-01-07,93,2997,53033000100
533,2019-01-07,136,2789,53033000200
586,2019-01-07,189,882,53033000300
740,2019-01-07,343,2390,53033000401
439,2019-01-07,42,1551,53033000402


In [22]:
devices_total_dates = devices_total["date"].sort_values().unique()
devices_total_dates.size

103

In [23]:
unique_days = devices_total["date"].drop_duplicates().sort_values().to_numpy()
days_diff = devices_total["date"].drop_duplicates().sort_values().diff().dt.days.to_numpy()

## Matrix devices

The "matrix" files depicts a census tract visit matrix summed across all census block groups in the census tract in the 1-week period. So, to rescale to get visits per device, one could divide each row by the devices number in the devices file. 

* Update: The "matrix" files didn't change much, except to get the correct week structure for end of 2019 through 2020.

In [24]:
devices_flow_dict = {}
for f in matrix_path.glob("*matrix.csv"):
    month_day, year= f.stem.removesuffix("matrix").split("_")
    month, day = month_day.split("-")
    day = day if len(day) == 2 else "0" + day
    file_date = "-".join([year, month, day])
    matrix_df = (
        pd.read_csv(f, index_col=0)
        .rename_axis("origin")
        .rename(columns=lambda x: int(x.lower().removeprefix("v")))
        .reset_index()
        .melt(id_vars="origin", var_name="destination", value_name="flow")
    )
    devices_flow_dict[file_date] = matrix_df

In [25]:
devices_flow = (
    pd.concat(devices_flow_dict, names=["date"])
    .droplevel(level=1)
    .reset_index()
    .astype({"origin": int, "destination": int})
    .query("origin in @id_list and destination in @id_list")
    .assign(
        date=lambda x: pd.to_datetime(x["date"]),
        origin=lambda x: x["origin"].map(id_tract_dict),
        destination=lambda x: x["destination"].map(id_tract_dict)
    )
    .sort_values(["date", "origin", "destination"])
    .merge(
        devices_total,
        how="inner",
        left_on=["date", "origin"],
        right_on=["date", "tract"]
    )
    .drop(columns="tract")
    .assign(
        flow_ratio=lambda x: x["flow"] / x["n_devices"]  # PROPORTION OF DEVICES!
    )
)
devices_flow.head(3)

,date,origin,destination,flow,id,n_devices,flow_ratio
0,2019-01-07,53033000100,53033000100,2821,93,2997,0.941275
1,2019-01-07,53033000100,53033000200,105,93,2997,0.035035
2,2019-01-07,53033000100,53033000300,13,93,2997,0.004338


In [27]:
# devices_flow.drop_duplicates(["date", "origin"]).shape[0] == 103 * n_tracts

In [33]:
devices_flow_dates = devices_flow["date"].unique()

In [ ]:
devices_flow.query("flow_ratio > 1 and origin != destination")

,date,origin,destination,flow,id,n_devices,flow_ratio
8670027,2020-02-03,53033005301,53033005302,1261,3,1133,1.112974
8824476,2020-02-10,53033005301,53033005302,1183,3,1052,1.124525
8978925,2020-02-17,53033005301,53033005302,1096,3,1000,1.096000
9133374,2020-02-24,53033005301,53033005302,1157,3,1060,1.091509
9417042,2020-03-02,53033032324,53033032309,1413,225,859,1.644936
...,...,...,...,...,...,...,...
15866496,2020-12-21,53033029504,53033030202,2006,283,1266,1.584518
15871930,2020-12-21,53033030202,53033026100,1048,311,828,1.265700
15871953,2020-12-21,53033030202,53033028300,1211,311,828,1.462560
15871962,2020-12-21,53033030202,53033028902,1637,311,828,1.977053


## Comp

The "comp" files contain the tract's number of devices completely at home in a day summed across the week. To get proportion completely at home, divide a tract's week's comp value by its "devices" value.

These two values (proportion completely at home AND median proportion of time at home) can serve as our measures of isolation and social distancing/avoidance, respectively.

~~Number of devices spending the entire day at home summed across all census block groups in the census tract for each day in the week. So, to estimate the percentage of devices spending all day at home that week, one can divide by the devices number from the devices file.~~

In [28]:
comp_dict = {}
for f in compmean_path.glob("*comp.csv"):
    month_day, year= f.stem.removesuffix("comp").split("_")
    month, day = month_day.split("-")
    day = day if len(day) == 2 else "0" + day
    file_date = "-".join([year, month, day])
    comp_dict[file_date] = pd.read_csv(
        f,
        index_col=0,
        names=["id", "n_devices_home"],
        header=0
    )

In [29]:
devices_home = (
    pd.concat(comp_dict, names=["date"])
    .reset_index()
    .assign(date=lambda x: pd.to_datetime(x["date"]))
    .query("id in @id_list")
    .merge(
        devices_total,
        how="inner",
        on=["date", "id"],
        validate="many_to_one"
        )
    .assign(
        tract=lambda x: x["id"].map(id_tract_dict),
        ratio_devices_home=lambda x: x["n_devices_home"] / x["n_devices"]
    )
    .sort_values(["date", "tract"])
)
devices_home.head()

,date,id,n_devices_home,n_devices,tract,ratio_devices_home
485,2019-01-07,93,1167,2997,53033000100,0.389389
526,2019-01-07,136,1134,2789,53033000200,0.406597
579,2019-01-07,189,357,882,53033000300,0.404762
731,2019-01-07,343,946,2390,53033000401,0.395816
434,2019-01-07,42,603,1551,53033000402,0.388781


In [30]:
# devices_home.drop_duplicates(["date", "id"]).shape[0] == 103 * n_tracts

In [34]:
devices_home_dates = devices_home["date"].unique()
print(np.setdiff1d(devices_home_dates, devices_total_dates))
print(np.setdiff1d(devices_home_dates, devices_flow_dates))

[]
[]


## Mean

~~Total number of minutes spent at home across all devices in the census tract. I calculated this by multiplying (for each census block group and day), the mean number of minutes at home for all devices by the number of devices, and then summing across all census block groups and days. So if one divides this number by the devices number in the devices file, it will produce the mean number of minutes spent at home in a given day in that week.~~

The "mean" files contain a tract value that you should divide by its "devices" value. After doing this, you'll have the tract's median proportion of time spent at home (which is a device-weighted average of the median proportion time at home from all block group days in the tract).

These two values (proportion completely at home AND median proportion of time at home) can serve as our measures of isolation and social distancing/avoidance, respectively.

# THIS IS THE PROPORTION OF MINUTES!!!

In [35]:
minutes_dict = {}
for f in compmean_path.glob("*mean.csv"):
    month_day, year= f.stem.removesuffix("mean").split("_")
    month, day = month_day.split("-")
    day = day if len(day) == 2 else "0" + day
    file_date = "-".join([year, month, day])
    minutes_dict[file_date] = pd.read_csv(
        f,
        index_col=0,
        names=["id", "minutes_home"],
        header=0
    )

In [36]:
devices_minutes_home = (
    pd.concat(minutes_dict, names=["date"])
    .reset_index()
    .assign(date=lambda x: pd.to_datetime(x["date"]))
    .query("id in @id_list")
    .merge(
        devices_total,
        how="inner",
        on=["date", "id"],
        validate="many_to_one"
    )
    .assign(
        perc_time_devices_home=lambda x: x["minutes_home"] / x["n_devices"],  # I think it could have a beter name
        tract=lambda x: x["id"].map(id_tract_dict)
    )
    .sort_values(["date", "tract"])
)
devices_minutes_home

,date,id,minutes_home,n_devices,tract,perc_time_devices_home
485,2019-01-07,93,270684,2997,53033000100,90.318318
526,2019-01-07,136,253157,2789,53033000200,90.769810
579,2019-01-07,189,82640,882,53033000300,93.696145
731,2019-01-07,343,212455,2390,53033000401,88.893305
434,2019-01-07,42,138686,1551,53033000402,89.417150
...,...,...,...,...,...,...
39345,2020-12-21,46,126881,1303,53033032402,97.376055
39670,2020-12-21,375,142148,1678,53033032601,84.712753
39367,2020-12-21,68,140854,1534,53033032602,91.821382
39524,2020-12-21,228,65826,676,53033032703,97.375740


In [39]:
# devices_minutes_home.drop_duplicates(["date", "id"]).shape[0] == 103 * n_tracts

In [38]:
devices_minutes_home_dates = devices_minutes_home["date"].unique()
print(np.setdiff1d(devices_minutes_home_dates , devices_total_dates))
print(np.setdiff1d(devices_minutes_home_dates , devices_flow_dates))
print(np.setdiff1d(devices_minutes_home_dates , devices_home_dates))

[]
[]
[]


## Agents


In [57]:
devices_tract = (
  devices_total.assign(n_devices=lambda x: x['n_devices'] / 7)  # 7 days per week
  .groupby('tract', as_index=False)
  .agg(
    mean_devices=('n_devices', 'mean'),
    std_devices=('n_devices', 'std')
  )
)
devices_tract.head()

,tract,mean_devices,std_devices
0,53033000100,350.757282,33.420076
1,53033000200,232.260749,124.728132
2,53033000300,188.074896,66.679328
3,53033000401,283.181692,24.879254
4,53033000402,174.029126,29.600341


In [ ]:
agents_tract = (
  devices_tract.assign(n_agents=lambda x: x['mean_devices'].astype(int))
  .loc[:, ['tract', 'n_agents']]
  .sort_values('tract')
)
agents_tract.head()

,tract,n_agents
0,53033000100,350
1,53033000200,232
2,53033000300,188
3,53033000401,283
4,53033000402,174


In [80]:
agents_tract.to_csv(data_path / "agents_tract.csv", index=False)
# agents_tract.to_json(data_path / "agents_tract.json", index=False)

## Devices completely at home

In [82]:
devices_home.head()

,date,id,n_devices_home,n_devices,tract,ratio_devices_home
485,2019-01-07,93,1167,2997,53033000100,0.389389
526,2019-01-07,136,1134,2789,53033000200,0.406597
579,2019-01-07,189,357,882,53033000300,0.404762
731,2019-01-07,343,946,2390,53033000401,0.395816
434,2019-01-07,42,603,1551,53033000402,0.388781


In [84]:
agents_home = (
  devices_home.rename(columns={'ratio_devices_home': 'prob_stay_home'})
  .loc[:, ['date', 'tract', 'prob_stay_home']]
  .sort_values(['date', 'tract'])
)
agents_home.head()

,date,tract,prob_stay_home
485,2019-01-07,53033000100,0.389389
526,2019-01-07,53033000200,0.406597
579,2019-01-07,53033000300,0.404762
731,2019-01-07,53033000401,0.395816
434,2019-01-07,53033000402,0.388781


In [85]:
agents_home.to_csv(data_path / "agents_home.csv", index=False)

## Percentage of time at home

In [87]:
agents_percentage_home = (
  devices_minutes_home.rename(columns={'perc_time_devices_home': 'percentage_time_home'})
  .loc[:, ['date', 'tract', 'percentage_time_home']]
  .sort_values(['date', 'tract'])
)
agents_percentage_home.head()

,date,tract,percentage_time_home
485,2019-01-07,53033000100,90.318318
526,2019-01-07,53033000200,90.769810
579,2019-01-07,53033000300,93.696145
731,2019-01-07,53033000401,88.893305
434,2019-01-07,53033000402,89.417150


In [88]:
agents_percentage_home.to_csv(data_path / "agents_percentage_home.csv", index=False)

## Flow

In [90]:
agents_flow = (
  devices_flow
  .loc[:, ['date', 'origin', 'destination', 'flow_ratio']]
  .sort_values(['date', 'origin', 'destination'])
)
agents_flow.head()

,date,origin,destination,flow_ratio
0,2019-01-07,53033000100,53033000100,0.941275
1,2019-01-07,53033000100,53033000200,0.035035
2,2019-01-07,53033000100,53033000300,0.004338
3,2019-01-07,53033000100,53033000401,0.012012
4,2019-01-07,53033000100,53033000402,0.006340


In [98]:
flow_path = data_path / "flow"
flow_path.mkdir(exist_ok=True)
for name, group in agents_flow.groupby("date"):
  date = name.strftime('%Y-%m-%d')
  group.drop(columns="date").to_csv(flow_path / f'agents_flow_{date}.csv', index=False)